In [1]:
#Import modules
%matplotlib inline

#handling data
import pandas as pd
import numpy as np

#import twitter credentials
from twitter_credentials import *

#handling information
import re
import json

#handling plots
import matplotlib.pyplot as plt

#for network creation
import networkx as nx

In [99]:
tweets_df = pd.read_json("tweets.txt", lines=True)

In [100]:
tweets_df.columns

Index(['contributors', 'coordinates', 'created_at', 'display_text_range',
       'entities', 'extended_entities', 'extended_tweet', 'favorite_count',
       'favorited', 'filter_level', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quote_count', 'quoted_status',
       'quoted_status_id', 'quoted_status_id_str', 'quoted_status_permalink',
       'reply_count', 'retweet_count', 'retweeted', 'retweeted_status',
       'source', 'text', 'timestamp_ms', 'truncated', 'user'],
      dtype='object')

In [101]:
tweets_df.head()

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,...,quoted_status_permalink,reply_count,retweet_count,retweeted,retweeted_status,source,text,timestamp_ms,truncated,user
0,NaN,None,2019-03-30 20:14:37,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,0,False,low,...,"{'url': 'https://t.co/RXBjmCJkif', 'expanded':...",0,0,False,{'created_at': 'Sat Mar 30 12:25:59 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @4fnan_zainal: Pls ban kids masa showtime a...,2019-03-30 20:14:37.028,False,"{'id': 925075656127365120, 'id_str': '92507565..."
1,NaN,None,2019-03-30 20:14:39,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,0,0,False,{'created_at': 'Sat Mar 30 00:44:39 +0000 2019...,"<a href=""http://twitter.com/download/android"" ...",RT @skipbolden: The Avengers (TV Series 1961–1...,2019-03-30 20:14:39.313,False,"{'id': 812752256214925312, 'id_str': '81275225..."
2,NaN,None,2019-03-30 20:14:41,NaN,"{'hashtags': [{'text': 'avengers', 'indices': ...",NaN,NaN,0,False,low,...,NaN,0,0,False,NaN,"<a href=""http://instagram.com"" rel=""nofollow"">...",Looking forward to the Avengers End Game 😍 #av...,2019-03-30 20:14:41.121,False,"{'id': 924977392535339010, 'id_str': '92497739..."
3,NaN,None,2019-03-30 20:14:41,NaN,"{'hashtags': [{'text': 'AvengeTheFallen', 'ind...",NaN,NaN,0,False,low,...,NaN,0,0,False,{'created_at': 'Sat Mar 30 17:15:19 +0000 2019...,"<a href=""http://twitter.com/download/iphone"" r...",RT @Hypable: 'Avengers: Endgame' posters have ...,2019-03-30 20:14:41.769,False,"{'id': 1377394638, 'id_str': '1377394638', 'na..."
4,NaN,None,2019-03-30 20:14:41,"[41, 98]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0,False,low,...,NaN,0,0,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",@MTV_iwan @theflairqueenx @BeckyLynchWWE I gue...,2019-03-30 20:14:41.879,False,"{'id': 1105100693147394048, 'id_str': '1105100..."


In [102]:
print(tweets_df["entities"][0])

{'hashtags': [], 'urls': [{'url': 'https://t.co/RXBjmCJkif', 'expanded_url': 'https://twitter.com/salce21_elijah/status/1111004620476821505', 'display_url': 'twitter.com/salce21_elijah…', 'indices': [68, 91]}], 'user_mentions': [{'screen_name': '4fnan_zainal', 'name': 'thepatotie', 'id': 938373003355635712, 'id_str': '938373003355635712', 'indices': [3, 16]}], 'symbols': []}


In [103]:
tweets_final = pd.DataFrame(columns = ["created_at", "id", "in_reply_to_screen_name", "in_reply_to_status_id", "in_reply_to_user_id",
                                      "retweeted_status", "quoted_status", "quoted_status_id", "user_mentions_screen_name", "user_mentions_id", 
                                       "text", "user_id", "screen_name", "followers_count"])

In [104]:
equal_columns = ["created_at", "id", "text"]
tweets_final[equal_columns] = tweets_df[equal_columns]

In [105]:
tweets_final["screen_name"] = tweets_df["user"].apply(lambda x: x["screen_name"])

In [106]:
tweets_final["user_id"] = tweets_df["user"].apply(lambda x: x["id"])

In [108]:
tweets_final["followers_count"] = tweets_df["user"].apply(lambda x: x["followers_count"])

In [117]:
tweets_final["user_mentions_id"] = tweets_df["entities"].apply(lambda x: x["user_mentions"])

In [112]:
tweets_final.head()

,created_at,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,retweeted_status,quoted_status,quoted_status_id,user_mentions_screen_name,user_mentions_id,text,user_id,screen_name,followers_count
0,2019-03-30 20:14:37,1112085741683638272,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': '4fnan_zainal', 'name': 'thep...",RT @4fnan_zainal: Pls ban kids masa showtime a...,925075656127365120,iRfansyamill,39
1,2019-03-30 20:14:39,1112085751267762177,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'skipbolden', 'name': 'Skip B...",RT @skipbolden: The Avengers (TV Series 1961–1...,812752256214925312,Brianlovesmovi,5906
2,2019-03-30 20:14:41,1112085758851059714,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[],Looking forward to the Avengers End Game 😍 #av...,924977392535339010,GadgetSwitched,29
3,2019-03-30 20:14:41,1112085761569054723,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'Hypable', 'name': 'Hypable.c...",RT @Hypable: 'Avengers: Endgame' posters have ...,1377394638,Christian_sexy7,266
4,2019-03-30 20:14:41,1112085762030358530,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'screen_name': 'MTV_iwan', 'name': 'sally wa...",@MTV_iwan @theflairqueenx @BeckyLynchWWE I gue...,1105100693147394048,QuincyWard15,28


In [96]:
tweets_df["user"][0]

{'id': 925075656127365120,
 'id_str': '925075656127365120',
 'name': 'Irfan Syamil',
 'screen_name': 'iRfansyamill',
 'location': None,
 'url': None,
 'description': 'nani?!',
 'translator_type': 'none',
 'protected': False,
 'verified': False,
 'followers_count': 39,
 'friends_count': 56,
 'listed_count': 0,
 'favourites_count': 5841,
 'statuses_count': 3058,
 'created_at': 'Mon Oct 30 19:03:18 +0000 2017',
 'utc_offset': None,
 'time_zone': None,
 'geo_enabled': False,
 'lang': 'en',
 'contributors_enabled': False,
 'is_translator': False,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': '',
 'profile_background_image_url_https': '',
 'profile_background_tile': False,
 'profile_link_color': '1DA1F2',
 'profile_sidebar_border_color': 'C0DEED',
 'profile_sidebar_fill_color': 'DDEEF6',
 'profile_text_color': '333333',
 'profile_use_background_image': True,
 'profile_image_url': 'http://pbs.twimg.com/profile_images/1079494377477615617/B1sjJsdO_normal.jpg',
 'profil